In [1]:
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (
    f1_score,
    recall_score,
    accuracy_score,
    precision_score,
    roc_auc_score,
    confusion_matrix,
    roc_curve,
    auc,
    make_scorer
)
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier

# Importações para reamostragem
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.metrics import geometric_mean_score # Importa G-Mean do imblearn

import matplotlib.pyplot as plt
import seaborn as sns

## Leitura dos Dados

In [3]:
## Leitura dos Dados
# Carrega o dataset
df = pd.read_csv('diabetes_binary_health_indicators_BRFSS2015.csv')

# Separa features (X) e target (y)
X = df.drop(columns=['Diabetes_binary'])
y = df['Diabetes_binary']

# Divide os dados em conjuntos de treino e teste, mantendo a proporção das classes
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

## Normalização min-max dos dados não binários e dos que possuem ordem 


In [4]:
columns_to_normalize = ["BMI","GenHlth","MentHlth","PhysHlth","Age","Education","Income"]
columns_to_pass_through = ["HighBP","HighChol","CholCheck","Stroke","HeartDiseaseorAttack","PhysActivity","Fruits","Veggies","HvyAlcoholConsump","AnyHealthcare","NoDocbcCost", "DiffWalk","Sex"]

preprocessor = ColumnTransformer(
    transformers=[
        ('minmax_scaler', MinMaxScaler(), columns_to_normalize),
        ('passthrough_features', 'passthrough', columns_to_pass_through)
    ])

## Métrica para escolha: f1-score na classe minoritária e Validação Cruzada

In [ ]:
### Métrica para escolha: f1-score na classe minoritária
# Define o scorer para GridSearchCV/RandomizedSearchCV, focando no f1-score da classe minoritária (1.0)
scorer = make_scorer(f1_score, pos_label=1.0)
# Define a estratégia de validação cruzada estratificada para manter a proporção das classes
cv_strategy = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

## Métricas para Avaliação dos Modelos

In [ ]:
# Função para avaliar e imprimir todas as métricas solicitadas
def evaluate_model(y_true, y_pred, y_prob=None, model_name="Modelo"):
    """
    Calcula e exibe métricas de classificação para um modelo.

    Args:
        y_true (array-like): Rótulos verdadeiros.
        y_pred (array-like): Previsões do modelo.
        y_prob (array-like, optional): Probabilidades previstas para a classe positiva.
                                       Necessário para ROC AUC e Curva ROC. Defaults to None.
        model_name (str, optional): Nome do modelo para exibição nos resultados. Defaults to "Modelo".
    """
    print(f"\n--- Métricas para {model_name} ---")

    # Matriz de Confusão
    cm = confusion_matrix(y_true, y_pred)
    print("\nMatriz de Confusão:")
    print(cm)
    plt.figure(figsize=(6, 4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Previsto 0', 'Previsto 1'], yticklabels=['Real 0', 'Real 1'])
    plt.title(f'Matriz de Confusão para {model_name}')
    plt.xlabel('Rótulo Previsto')
    plt.ylabel('Rótulo Real')
    plt.show()

    # Métricas por classe
    # precision_score, recall_score, f1_score com average=None retornam os valores por classe
    precision_class_0, precision_class_1 = precision_score(y_true, y_pred, average=None)
    recall_class_0, recall_class_1 = recall_score(y_true, y_pred, average=None)
    f1_class_0, f1_class_1 = f1_score(y_true, y_pred, average=None)

    print(f"\nMétricas para a Classe 0 (Majoritária):")
    print(f"  Precisão: {precision_class_0:.4f}")
    print(f"  Recall: {recall_class_0:.4f}")
    print(f"  F1-Score: {f1_class_0:.4f}")

    print(f"\nMétricas para a Classe 1 (Minoritária):")
    print(f"  Precisão: {precision_class_1:.4f}")
    print(f"  Recall: {recall_class_1:.4f}")
    print(f"  F1-Score: {f1_class_1:.4f}")

    # Métricas gerais
    accuracy = accuracy_score(y_true, y_pred)
    # f1_score com average='macro' calcula a média não ponderada do f1-score por classe
    f1_macro = f1_score(y_true, y_pred, average='macro')
    # G-Mean para classes desbalanceadas
    g_mean = geometric_mean_score(y_true, y_pred)


    print(f"\nMétricas Gerais:")
    print(f"  Acurácia: {accuracy:.4f}")
    print(f"  F1-Score (Macro): {f1_macro:.4f}")
    print(f"  G-Mean: {g_mean:.4f}")

    # ROC AUC e Curva ROC
    # Verifica se y_prob foi fornecido e se é uma classificação binária
    if y_prob is not None and len(np.unique(y_true)) == 2:
        roc_auc = roc_auc_score(y_true, y_prob)
        fpr, tpr, thresholds = roc_curve(y_true, y_prob)
        print(f"  ROC AUC: {roc_auc:.4f}")

        plt.figure(figsize=(8, 6))
        plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'Curva ROC (área = {roc_auc:.2f})')
        plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--') # Linha de classificação aleatória
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('Taxa de Falsos Positivos (FPR)')
        plt.ylabel('Taxa de Verdadeiros Positivos (TPR)')
        plt.title(f'Curva Característica de Operação do Receptor (ROC) para {model_name}')
        plt.legend(loc="lower right")
        plt.show()
    else:
        print("  ROC AUC e Curva ROC não aplicáveis (não é classificação binária ou y_prob não fornecido).")

## Treinamento para árvore de decisão

### Ajuste de Hiperparâmetro com Quatro Tipos de Métodos de Reamostragem

In [5]:
# Pipeline básico (sem reamostragem)
pipeline_no_sampling = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', DecisionTreeClassifier(random_state=42))
])

# Pipeline com SMOTE
pipeline_smote = ImbPipeline([
    ('preprocessor', preprocessor),
    ('sampler', SMOTE(random_state=42)),
    ('classifier', DecisionTreeClassifier(random_state=42))
])

# Pipeline com ADASYN
pipeline_adasyn = ImbPipeline([
    ('preprocessor', preprocessor),
    ('sampler', ADASYN(random_state=42)),
    ('classifier', DecisionTreeClassifier(random_state=42))
])

# Pipeline com Undersampling seguido por SMOTE
pipeline_under_smote = ImbPipeline([
    ('preprocessor', preprocessor),
    ('under_sampler', RandomUnderSampler(random_state=42)),
    ('over_sampler', SMOTE(random_state=42)),
    ('classifier', DecisionTreeClassifier(random_state=42))
])

# Pipeline com Undersampling seguido por ADASYN
pipeline_under_adasyn = ImbPipeline([
    ('preprocessor', preprocessor),
    ('under_sampler', RandomUnderSampler(random_state=42)),
    ('over_sampler', ADASYN(random_state=42)),
    ('classifier', DecisionTreeClassifier(random_state=42))
])

# Lista de pipelines para experimentação com Árvore de Decisão
pipelines_dt = {
    'No Sampling': pipeline_no_sampling,
    'SMOTE': pipeline_smote,
    'ADASYN': pipeline_adasyn,
    'Under+SMOTE': pipeline_under_smote,
    'Under+ADASYN': pipeline_under_adasyn
}

### Hiperparâmetros: Número de Vizinhos (para os métodos SMOTE e ADASYN), Proporção da classe majoritária (undersampling), Altura de Árvore e Índice de Impureza

### Resultados: sem otimização de hiperparâmetros (Árvore de Decisão)

In [6]:
print("\n--- Desempenho Inicial dos Pipelines (Árvore de Decisão - sem otimização de hiperparâmetros) ---")
for name, pipeline in pipelines_dt.items():
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    y_prob = pipeline.predict_proba(X_test)[:, 1] # Obtém probabilidades para ROC AUC
    evaluate_model(y_test, y_pred, y_prob, f"Árvore de Decisão - {name}")


--- Desempenho Inicial dos Pipelines (sem otimização de hiperparâmetros) ---

No Sampling:
              precision    recall  f1-score   support

         0.0       0.89      0.88      0.88     54583
         1.0       0.30      0.33      0.31      8837

    accuracy                           0.80     63420
   macro avg       0.59      0.60      0.60     63420
weighted avg       0.81      0.80      0.80     63420

F1-Score (macro): 0.5966
ROC AUC: 0.6005

SMOTE:
              precision    recall  f1-score   support

         0.0       0.89      0.87      0.88     54583
         1.0       0.29      0.32      0.30      8837

    accuracy                           0.80     63420
   macro avg       0.59      0.60      0.59     63420
weighted avg       0.80      0.80      0.80     63420

F1-Score (macro): 0.5911
ROC AUC: 0.5954

ADASYN:
              precision    recall  f1-score   support

         0.0       0.89      0.87      0.88     54583
         1.0       0.28      0.31      0.30   

### Resultados: com ajuste de hiperparâmetros (Sem reamostragem - Árvore de Decisão)


In [ ]:
param_grid_dt = {
    'classifier__max_depth': [3, 5, 7, 9, 11, 13, 14, None], # Altura da árvore
    'classifier__criterion': ['gini', 'entropy'], # Índice de pureza
}

In [8]:
# --- Otimização para Sem Reamostragem (Árvore de Decisão) ---
print("\nOtimizando Pipeline Sem Reamostragem (Árvore de Decisão)...")
grid_dt_no_sampling = RandomizedSearchCV(pipeline_no_sampling, param_grid_dt, cv=cv_strategy,
                          scoring=scorer, n_jobs=-1, verbose=1, n_iter=10, random_state=42)

grid_dt_no_sampling.fit(X_train, y_train)

print(f"Melhores parâmetros Sem Reamostragem (Árvore de Decisão): {grid_dt_no_sampling.best_params_}")
y_pred_tuned = grid_dt_no_sampling.best_estimator_.predict(X_test)
y_prob_tuned = grid_dt_no_sampling.best_estimator_.predict_proba(X_test)[:, 1]
evaluate_model(y_test, y_pred_tuned, y_prob_tuned, "Árvore de Decisão - Sem Reamostragem (Otimizado)")


Otimizando Pipeline Sem Reamostragem...
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Melhores parâmetros Sem Reamostragem: {'classifier__criterion': 'entropy', 'classifier__max_depth': None}
Relatório de Classificação Sem Reamostragem:
              precision    recall  f1-score   support

         0.0       0.89      0.88      0.88     54583
         1.0       0.30      0.32      0.31      8837

    accuracy                           0.80     63420
   macro avg       0.60      0.60      0.60     63420
weighted avg       0.81      0.80      0.81     63420

F1-Score (macro): 0.5989
ROC AUC: 0.6015


### Resultados: com ajuste de hiperparâmetros (SMOTE - Árvore de Decisão)


In [ ]:
param_grid_smote_dt = {
    'sampler__k_neighbors': [3, 5, 7, 10],  # k_neighbors para SMOTE
    'classifier__max_depth': [3, 5, 7, 10, None], # Altura da árvore
    'classifier__criterion': ['gini', 'entropy'], # Índice de pureza
}

In [7]:
param_grid_smote_dt = {
    'sampler__k_neighbors': [3, 5, 7, 10],  # k_neighbors para SMOTE
    'classifier__max_depth': [3, 5, 7, 10, None], # Altura da árvore
    'classifier__criterion': ['gini', 'entropy'], # Índice de pureza
}

# --- Otimização para SMOTE (Árvore de Decisão) ---
print("\nOtimizando Pipeline com SMOTE (Árvore de Decisão)...")
grid_dt_smote = RandomizedSearchCV(pipeline_smote, param_grid_smote_dt, n_iter=10, cv=cv_strategy,
                          scoring=scorer, n_jobs=-1, verbose=1, random_state=42)

grid_dt_smote.fit(X_train, y_train)

print(f"Melhores parâmetros para SMOTE (Árvore de Decisão): {grid_dt_smote.best_params_}")
y_pred_smote_tuned = grid_dt_smote.best_estimator_.predict(X_test)
y_prob_smote_tuned = grid_dt_smote.best_estimator_.predict_proba(X_test)[:, 1]
evaluate_model(y_test, y_pred_smote_tuned, y_prob_smote_tuned, "Árvore de Decisão - SMOTE (Otimizado)")


Otimizando Pipeline com SMOTE...
Fitting 5 folds for each of 10 candidates, totalling 50 fits


### Resultados: com ajuste de hiperparâmetros (ADASYN - Árvore de Decisão)


In [ ]:
param_grid_adasyn_dt = {
    'sampler__n_neighbors': [3, 5, 7, 10], # n_neighbors para ADASYN
    'classifier__max_depth': [3, 5, 7, 10, None],
    'classifier__criterion': ['gini', 'entropy']
}

In [ ]:
# --- Otimização para ADASYN (Árvore de Decisão) ---
print("\nOtimizando Pipeline com ADASYN (Árvore de Decisão)...")
grid_dt_adasyn = RandomizedSearchCV(pipeline_adasyn, param_grid_adasyn_dt, n_iter=10, cv=cv_strategy,
                          scoring=scorer, n_jobs=-1, verbose=1, random_state=42)

grid_dt_adasyn.fit(X_train, y_train)

print(f"Melhores parâmetros para ADASYN (Árvore de Decisão): {grid_dt_adasyn.best_params_}")
y_pred_adasyn_tuned = grid_dt_adasyn.best_estimator_.predict(X_test)
y_prob_adasyn_tuned = grid_dt_adasyn.best_estimator_.predict_proba(X_test)[:, 1]
evaluate_model(y_test, y_pred_adasyn_tuned, y_prob_adasyn_tuned, "Árvore de Decisão - ADASYN (Otimizado)")


### Resultados: com ajuste de hiperparâmetros (SMOTE + Undersampling - Árvore de Decisão)


In [ ]:
param_grid_under_smote_dt = {
    'under_sampler__sampling_strategy': [0.5, 0.7, 0.9], # Proporção da classe majoritária após undersampling
    'over_sampler__k_neighbors': [3, 5, 7, 10],
    'classifier__max_depth': [3, 5, 7, 10, None],
    'classifier__criterion': ['gini', 'entropy']
}

In [ ]:
# --- Otimização para SMOTE + Undersampling (Árvore de Decisão) ---
print("\nOtimizando Pipeline com SMOTE + Undersampling (Árvore de Decisão)...")
grid_dt_under_smote = RandomizedSearchCV(pipeline_under_smote, param_grid_under_smote_dt, n_iter=10, cv=cv_strategy,
                          scoring=scorer, n_jobs=-1, verbose=1, random_state=42)

grid_dt_under_smote.fit(X_train, y_train)

print(f"Melhores parâmetros para SMOTE + Undersampling (Árvore de Decisão): {grid_dt_under_smote.best_params_}")
y_pred_under_smote_tuned = grid_dt_under_smote.best_estimator_.predict(X_test)
y_prob_under_smote_tuned = grid_dt_under_smote.best_estimator_.predict_proba(X_test)[:, 1]
evaluate_model(y_test, y_pred_under_smote_tuned, y_prob_under_smote_tuned, "Árvore de Decisão - Under+SMOTE (Otimizado)")

### Resultados: com ajuste de hiperparâmetros (ADASYN + Undersampling - Árvore de Decisão)


In [ ]:
param_grid_under_adasyn_dt = {
    'under_sampler__sampling_strategy': [0.5, 0.7, 0.9], # Proporção da classe majoritária após undersampling
    'over_sampler__n_neighbors': [3, 5, 7, 10],
    'classifier__max_depth': [3, 5, 7, 10, None],
    'classifier__criterion': ['gini', 'entropy']
}

In [ ]:
# --- Otimização para ADASYN + Undersampling (Árvore de Decisão) ---
print("\nOtimizando Pipeline com ADASYN + Undersampling (Árvore de Decisão)...")
grid_dt_under_adasyn = RandomizedSearchCV(pipeline_under_adasyn, param_grid_under_adasyn_dt, n_iter=10, cv=cv_strategy,
                          scoring=scorer, n_jobs=-1, verbose=1, random_state=42)

grid_dt_under_adasyn.fit(X_train, y_train)

print(f"Melhores parâmetros para ADASYN + Undersampling (Árvore de Decisão): {grid_dt_under_adasyn.best_params_}")
y_pred_under_adasyn_tuned = grid_dt_under_adasyn.best_estimator_.predict(X_test)
y_prob_under_adasyn_tuned = grid_dt_under_adasyn.best_estimator_.predict_proba(X_test)[:, 1]
evaluate_model(y_test, y_pred_under_adasyn_tuned, y_prob_under_adasyn_tuned, "Árvore de Decisão - Under+ADASYN (Otimizado)")

## Treinamento para AdaBoot

### Ajuste de Hiperparâmetro com Quatro Tipos de Métodos de Reamostragem

In [16]:
# Pipeline básico (sem reamostragem) para AdaBoost
pipeline_no_sampling_ada = Pipeline([
    ('preprocessor', preprocessor),
    ('adaboost', AdaBoostClassifier(estimator=DecisionTreeClassifier(random_state=42), random_state=42))
])

# Pipeline com SMOTE para AdaBoost
pipeline_smote_ada = ImbPipeline([
    ('preprocessor', preprocessor),
    ('sampler', SMOTE(random_state=42)),
    ('adaboost', AdaBoostClassifier(estimator=DecisionTreeClassifier(random_state=42), random_state=42))
])

# Pipeline com ADASYN para AdaBoost
pipeline_adasyn_ada = ImbPipeline([
    ('preprocessor', preprocessor),
    ('sampler', ADASYN(random_state=42)),
    ('adaboost', AdaBoostClassifier(estimator=DecisionTreeClassifier(random_state=42), random_state=42))
])

# Pipeline com Undersampling seguido por SMOTE para AdaBoost
pipeline_under_smote_ada = ImbPipeline([
    ('preprocessor', preprocessor),
    ('under_sampler', RandomUnderSampler(random_state=42)),
    ('over_sampler', SMOTE(random_state=42)),
    ('adaboost', AdaBoostClassifier(estimator=DecisionTreeClassifier(random_state=42), random_state=42))
])

# Pipeline com Undersampling seguido por ADASYN para AdaBoost
pipeline_under_adasyn_ada = ImbPipeline([
    ('preprocessor', preprocessor),
    ('under_sampler', RandomUnderSampler(random_state=42)),
    ('over_sampler', ADASYN(random_state=42)),
    ('adaboost', AdaBoostClassifier(estimator=DecisionTreeClassifier(random_state=42), random_state=42))
])

# Lista de pipelines para experimentação (AdaBoost)
pipelines_ada = {
    'No Sampling': pipeline_no_sampling_ada,
    'SMOTE': pipeline_smote_ada,
    'ADASYN': pipeline_adasyn_ada,
    'Under+SMOTE': pipeline_under_smote_ada,
    'Under+ADASYN': pipeline_under_adasyn_ada
}

### Resultados: sem otimização de hiperparâmetros (AdaBoost)


In [13]:
print("\n--- Desempenho Inicial dos Pipelines (AdaBoost - sem otimização de hiperparâmetros) ---")
for name, pipeline in pipelines_ada.items():
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    y_prob = pipeline.predict_proba(X_test)[:, 1]
    evaluate_model(y_test, y_pred, y_prob, f"AdaBoost - {name}")


--- Desempenho Inicial dos Pipelines (sem otimização de hiperparâmetros) ---

No Sampling:
              precision    recall  f1-score   support

         0.0       0.88      0.97      0.92     54583
         1.0       0.52      0.18      0.27      8837

    accuracy                           0.86     63420
   macro avg       0.70      0.58      0.60     63420
weighted avg       0.83      0.86      0.83     63420

F1-Score (macro): 0.5975
ROC AUC: 0.5778

SMOTE:
              precision    recall  f1-score   support

         0.0       0.94      0.75      0.84     54583
         1.0       0.32      0.72      0.44      8837

    accuracy                           0.75     63420
   macro avg       0.63      0.73      0.64     63420
weighted avg       0.86      0.75      0.78     63420

F1-Score (macro): 0.6400
ROC AUC: 0.7348

ADASYN:
              precision    recall  f1-score   support

         0.0       0.94      0.75      0.84     54583
         1.0       0.32      0.72      0.44   

### Resultados: com ajuste de hiperparâmetros (Sem reamostragem - AdaBoost)


In [ ]:
param_grid_ada = {
    'adaboost__n_estimators': [50, 100, 200],  # Número de estimadores fracos (árvores)
    'adaboost__learning_rate': [0.01, 0.1, 1.0],  # Contribuição de cada estimador
    'adaboost__estimator__max_depth': [1, 2, 3],  # Profundidade máxima da árvore no estimador base
    'adaboost__estimator__criterion': ['gini', 'entropy'] # Critério de pureza para o estimador base
}

In [18]:
print("\nOtimizando Pipeline Sem Reamostragem (AdaBoost)...")
grid_ada_no_sampling = RandomizedSearchCV(pipeline_no_sampling_ada, param_grid_ada, cv=cv_strategy,
                          scoring=scorer, n_jobs=-1, verbose=1, n_iter=10, random_state=42)

grid_ada_no_sampling.fit(X_train, y_train)

print(f"Melhores parâmetros Sem Reamostragem (AdaBoost): {grid_ada_no_sampling.best_params_}")
y_pred_tuned = grid_ada_no_sampling.best_estimator_.predict(X_test)
y_prob_tuned = grid_ada_no_sampling.best_estimator_.predict_proba(X_test)[:, 1]
evaluate_model(y_test, y_pred_tuned, y_prob_tuned, "AdaBoost - Sem Reamostragem (Otimizado)")



Otimizando Pipeline Sem Reamostragem...
Fitting 5 folds for each of 54 candidates, totalling 270 fits
Melhores parâmetros Sem Reamostragem: {'adaboost__estimator__criterion': 'gini', 'adaboost__estimator__max_depth': 3, 'adaboost__learning_rate': 0.1, 'adaboost__n_estimators': 200}
Relatório de Classificação Sem Reamostragem:
              precision    recall  f1-score   support

         0.0       0.88      0.97      0.92     54583
         1.0       0.52      0.20      0.29      8837

    accuracy                           0.86     63420
   macro avg       0.70      0.58      0.61     63420
weighted avg       0.83      0.86      0.84     63420

F1-Score (macro): 0.6063
ROC AUC: 0.5850


### Resultados: com ajuste de hiperparâmetros (SMOTE - AdaBoost)

In [ ]:
param_grid_smote_ada = {
    'sampler__k_neighbors': [3, 5, 7, 10], # k_neighbors para SMOTE
    'adaboost__n_estimators': [50, 100, 200],
    'adaboost__learning_rate': [0.01, 0.1, 1.0],
    'adaboost__estimator__max_depth': [1, 2, 3],
    'adaboost__estimator__criterion': ['gini', 'entropy']
}

In [23]:
# --- Otimização para SMOTE (AdaBoost) ---
print("\nOtimizando Pipeline com SMOTE (AdaBoost)...")
grid_ada_smote = RandomizedSearchCV(pipeline_smote_ada, param_grid_smote_ada, n_iter=10, cv=cv_strategy,
                          scoring=scorer, n_jobs=-1, verbose=1, random_state=42)

grid_ada_smote.fit(X_train, y_train)

print(f"Melhores parâmetros para SMOTE (AdaBoost): {grid_ada_smote.best_params_}")
y_pred_smote_tuned = grid_ada_smote.best_estimator_.predict(X_test)
y_prob_smote_tuned = grid_ada_smote.best_estimator_.predict_proba(X_test)[:, 1]
evaluate_model(y_test, y_pred_smote_tuned, y_prob_smote_tuned, "AdaBoost - SMOTE (Otimizado)")


Otimizando Pipeline Sem Reamostragem...
Fitting 5 folds for each of 54 candidates, totalling 270 fits


KeyboardInterrupt: 

### Resultados: com ajuste de hiperparâmetros (ADASYN - AdaBoost)


In [ ]:
param_grid_adasyn_ada = {
    'sampler__n_neighbors': [3, 5, 7, 10], # n_neighbors para ADASYN
    'adaboost__n_estimators': [50, 100, 200],
    'adaboost__learning_rate': [0.01, 0.1, 1.0],
    'adaboost__estimator__max_depth': [1, 2, 3],
    'adaboost__estimator__criterion': ['gini', 'entropy']
}

In [ ]:
# --- Otimização para ADASYN (AdaBoost) ---
print("\nOtimizando Pipeline com ADASYN (AdaBoost)...")
grid_ada_adasyn = RandomizedSearchCV(pipeline_adasyn_ada, param_grid_adasyn_ada, n_iter=10, cv=cv_strategy,
                          scoring=scorer, n_jobs=-1, verbose=1, random_state=42)

grid_ada_adasyn.fit(X_train, y_train)

print(f"Melhores parâmetros para ADASYN (AdaBoost): {grid_ada_adasyn.best_params_}")
y_pred_adasyn_tuned = grid_ada_adasyn.best_estimator_.predict(X_test)
y_prob_adasyn_tuned = grid_ada_adasyn.best_estimator_.predict_proba(X_test)[:, 1]
evaluate_model(y_test, y_pred_adasyn_tuned, y_prob_adasyn_tuned, "AdaBoost - ADASYN (Otimizado)")


### Resultados: com ajuste de hiperparâmetros (SMOTE + Undersampling - AdaBoost)


In [ ]:
param_grid_under_smote_ada = {
    'under_sampler__sampling_strategy': [0.5, 0.7, 0.9],
    'over_sampler__k_neighbors': [3, 5, 7, 10],
    'adaboost__n_estimators': [50, 100, 200],
    'adaboost__learning_rate': [0.01, 0.1, 1.0],
    'adaboost__estimator__max_depth': [1, 2, 3],
    'adaboost__estimator__criterion': ['gini', 'entropy']
}

In [ ]:
# --- Otimização para SMOTE + Undersampling (AdaBoost) ---
print("\nOtimizando Pipeline com SMOTE + Undersampling (AdaBoost)...")
grid_ada_under_smote = RandomizedSearchCV(pipeline_under_smote_ada, param_grid_under_smote_ada, n_iter=10, cv=cv_strategy,
                          scoring=scorer, n_jobs=-1, verbose=1, random_state=42)

grid_ada_under_smote.fit(X_train, y_train)

print(f"Melhores parâmetros para SMOTE + Undersampling (AdaBoost): {grid_ada_under_smote.best_params_}")
y_pred_under_smote_tuned = grid_ada_under_smote.best_estimator_.predict(X_test)
y_prob_under_smote_tuned = grid_ada_under_smote.best_estimator_.predict_proba(X_test)[:, 1]
evaluate_model(y_test, y_pred_under_smote_tuned, y_prob_under_smote_tuned, "AdaBoost - Under+SMOTE (Otimizado)")

### Resultados: com ajuste de hiperparâmetros (ADASYN + Undersampling - AdaBoost)


In [ ]:
param_grid_under_adasyn_ada = {
    'under_sampler__sampling_strategy': [0.5, 0.7, 0.9],
    'over_sampler__n_neighbors': [3, 5, 7, 10],
    'adaboost__n_estimators': [50, 100, 200],
    'adaboost__learning_rate': [0.01, 0.1, 1.0],
    'adaboost__estimator__max_depth': [1, 2, 3],
    'adaboost__estimator__criterion': ['gini', 'entropy']
}

In [ ]:
# --- Otimização para ADASYN + Undersampling (AdaBoost) ---
print("\nOtimizando Pipeline com ADASYN + Undersampling (AdaBoost)...")
grid_ada_under_adasyn = RandomizedSearchCV(pipeline_under_adasyn_ada, param_grid_under_adasyn_ada, n_iter=10, cv=cv_strategy,
                          scoring=scorer, n_jobs=-1, verbose=1, random_state=42)

grid_ada_under_adasyn.fit(X_train, y_train)

print(f"Melhores parâmetros para ADASYN + Undersampling (AdaBoost): {grid_ada_under_adasyn.best_params_}")
y_pred_under_adasyn_tuned = grid_ada_under_adasyn.best_estimator_.predict(X_test)
y_prob_under_adasyn_tuned = grid_ada_under_adasyn.best_estimator_.predict_proba(X_test)[:, 1]
evaluate_model(y_test, y_pred_under_adasyn_tuned, y_prob_under_adasyn_tuned, "AdaBoost - Under+ADASYN (Otimizado)")

## Treinamento para GradientBoostingClassifier

### Ajuste de Hiperparâmetro com Quatro Tipos de Métodos de Reamostragem


In [20]:
# Pipeline básico (sem reamostragem) para GradientBoosting
pipeline_no_sampling_gb = Pipeline([
    ('preprocessor', preprocessor),
    ('gradientboost', GradientBoostingClassifier(random_state=42))
])

# Pipeline com SMOTE para GradientBoosting
pipeline_smote_gb = ImbPipeline([
    ('preprocessor', preprocessor),
    ('sampler', SMOTE(random_state=42)),
    ('gradientboost', GradientBoostingClassifier(random_state=42))
])

# Pipeline com ADASYN para GradientBoosting
pipeline_adasyn_gb = ImbPipeline([
    ('preprocessor', preprocessor),
    ('sampler', ADASYN(random_state=42)),
    ('gradientboost', GradientBoostingClassifier(random_state=42))
])

# Pipeline com Undersampling seguido por SMOTE para GradientBoosting
pipeline_under_smote_gb = ImbPipeline([
    ('preprocessor', preprocessor),
    ('under_sampler', RandomUnderSampler(random_state=42)),
    ('over_sampler', SMOTE(random_state=42)),
    ('gradientboost', GradientBoostingClassifier(random_state=42))
])

# Pipeline com Undersampling seguido por ADASYN para GradientBoosting
pipeline_under_adasyn_gb = ImbPipeline([
    ('preprocessor', preprocessor),
    ('under_sampler', RandomUnderSampler(random_state=42)),
    ('over_sampler', ADASYN(random_state=42)),
    ('gradientboost', GradientBoostingClassifier(random_state=42))
])

# Lista de pipelines para experimentação com GradientBoosting
pipelines_gb = {
    'No Sampling': pipeline_no_sampling_gb,
    'SMOTE': pipeline_smote_gb,
    'ADASYN': pipeline_adasyn_gb,
    'Under+SMOTE': pipeline_under_smote_gb,
    'Under+ADASYN': pipeline_under_adasyn_gb
}

### Resultados: sem otimização de hiperparâmetros (Gradient Boosting)


In [22]:
print("\n--- Desempenho Inicial dos Pipelines (Gradient Boosting - sem otimização de hiperparâmetros) ---")
for name, pipeline in pipelines_gb.items():
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    y_prob = pipeline.predict_proba(X_test)[:, 1]
    evaluate_model(y_test, y_pred, y_prob, f"Gradient Boosting - {name}")


--- Desempenho Inicial dos Pipelines (sem otimização de hiperparâmetros) ---

No Sampling:
              precision    recall  f1-score   support

         0.0       0.88      0.98      0.93     54583
         1.0       0.55      0.16      0.25      8837

    accuracy                           0.86     63420
   macro avg       0.71      0.57      0.59     63420
weighted avg       0.83      0.86      0.83     63420

F1-Score (macro): 0.5879
ROC AUC: 0.5703

SMOTE:
              precision    recall  f1-score   support

         0.0       0.91      0.88      0.90     54583
         1.0       0.41      0.49      0.45      8837

    accuracy                           0.83     63420
   macro avg       0.66      0.69      0.67     63420
weighted avg       0.84      0.83      0.84     63420

F1-Score (macro): 0.6728
ROC AUC: 0.6884

ADASYN:
              precision    recall  f1-score   support

         0.0       0.91      0.88      0.90     54583
         1.0       0.40      0.49      0.44   

### Resultados: com ajuste de hiperparâmetros (Sem reamostragem - Gradient Boosting)


In [24]:
param_grid_gb = {
    'gradientboost__n_estimators': [50, 100, 200],
    'gradientboost__learning_rate': [0.01, 0.1, 0.2],
    'gradientboost__max_depth': [3, 5, 7],
    'gradientboost__min_samples_split': [2, 5],
    'gradientboost__min_samples_leaf': [1, 3],
    'gradientboost__subsample': [0.8, 1.0],
}

In [ ]:
# --- Otimização para Sem Reamostragem (Gradient Boosting) ---
print("\nOtimizando Pipeline Sem Reamostragem (Gradient Boosting)...")
grid_gb_no_sampling = RandomizedSearchCV(pipeline_no_sampling_gb, param_grid_gb, cv=cv_strategy,
                          scoring=scorer, n_jobs=-1, verbose=1, n_iter=10, random_state=42)

grid_gb_no_sampling.fit(X_train, y_train)

print(f"Melhores parâmetros Sem Reamostragem (Gradient Boosting): {grid_gb_no_sampling.best_params_}")
y_pred_tuned = grid_gb_no_sampling.best_estimator_.predict(X_test)
y_prob_tuned = grid_gb_no_sampling.best_estimator_.predict_proba(X_test)[:, 1]
evaluate_model(y_test, y_pred_tuned, y_prob_tuned, "Gradient Boosting - Sem Reamostragem (Otimizado)")

### Resultados: com ajuste de hiperparâmetros (SMOTE - Gradient Boosting)


In [ ]:
param_grid_smote_gb = {
    'sampler__k_neighbors': [3, 5, 7, 10],
    'gradientboost__n_estimators': [50, 100, 200],
    'gradientboost__learning_rate': [0.01, 0.1, 0.2],
    'gradientboost__max_depth': [3, 5, 7],
    'gradientboost__min_samples_split': [2, 5],
    'gradientboost__min_samples_leaf': [1, 3],
    'gradientboost__subsample': [0.8, 1.0],
}

In [ ]:
# --- Otimização para SMOTE (Gradient Boosting) ---
print("\nOtimizando Pipeline com SMOTE (Gradient Boosting)...")
grid_gb_smote = RandomizedSearchCV(pipeline_smote_gb, param_grid_smote_gb, n_iter=10, cv=cv_strategy,
                          scoring=scorer, n_jobs=-1, verbose=1, random_state=42)

grid_gb_smote.fit(X_train, y_train)

print(f"Melhores parâmetros para SMOTE (Gradient Boosting): {grid_gb_smote.best_params_}")
y_pred_smote_tuned = grid_gb_smote.best_estimator_.predict(X_test)
y_prob_smote_tuned = grid_gb_smote.best_estimator_.predict_proba(X_test)[:, 1]
evaluate_model(y_test, y_pred_smote_tuned, y_prob_smote_tuned, "Gradient Boosting - SMOTE (Otimizado)")

### Resultados: com ajuste de hiperparâmetros (ADASYN - Gradient Boosting)


In [ ]:
param_grid_adasyn_gb = {
    'sampler__n_neighbors': [3, 5, 7, 10],
    'gradientboost__n_estimators': [50, 100, 200],
    'gradientboost__learning_rate': [0.01, 0.1, 0.2],
    'gradientboost__max_depth': [3, 5, 7],
    'gradientboost__min_samples_split': [2, 5],
    'gradientboost__min_samples_leaf': [1, 3],
    'gradientboost__subsample': [0.8, 1.0],
}

In [ ]:
# --- Otimização para ADASYN (Gradient Boosting) ---
print("\nOtimizando Pipeline com ADASYN (Gradient Boosting)...")
grid_gb_adasyn = RandomizedSearchCV(pipeline_adasyn_gb, param_grid_adasyn_gb, n_iter=10, cv=cv_strategy,
                          scoring=scorer, n_jobs=-1, verbose=1, random_state=42)

grid_gb_adasyn.fit(X_train, y_train)

print(f"Melhores parâmetros para ADASYN (Gradient Boosting): {grid_gb_adasyn.best_params_}")
y_pred_adasyn_tuned = grid_gb_adasyn.best_estimator_.predict(X_test)
y_prob_adasyn_tuned = grid_gb_adasyn.best_estimator_.predict_proba(X_test)[:, 1]
evaluate_model(y_test, y_pred_adasyn_tuned, y_prob_adasyn_tuned, "Gradient Boosting - ADASYN (Otimizado)")


### Resultados: com ajuste de hiperparâmetros (SMOTE + Undersampling - Gradient Boosting)


In [ ]:
param_grid_under_smote_gb = {
    'under_sampler__sampling_strategy': [0.5, 0.7, 0.9],
    'over_sampler__k_neighbors': [3, 5, 7, 10],
    'gradientboost__n_estimators': [50, 100, 200],
    'gradientboost__learning_rate': [0.01, 0.1, 0.2],
    'gradientboost__max_depth': [3, 5, 7],
    'gradientboost__min_samples_split': [2, 5],
    'gradientboost__min_samples_leaf': [1, 3],
    'gradientboost__subsample': [0.8, 1.0],
}

In [ ]:
# --- Otimização para SMOTE + Undersampling (Gradient Boosting) ---
print("\nOtimizando Pipeline com SMOTE + Undersampling (Gradient Boosting)...")
grid_gb_under_smote = RandomizedSearchCV(pipeline_under_smote_gb, param_grid_under_smote_gb, n_iter=10, cv=cv_strategy,
                          scoring=scorer, n_jobs=-1, verbose=1, random_state=42)

grid_gb_under_smote.fit(X_train, y_train)

print(f"Melhores parâmetros para SMOTE + Undersampling (Gradient Boosting): {grid_gb_under_smote.best_params_}")
y_pred_under_smote_tuned = grid_gb_under_smote.best_estimator_.predict(X_test)
y_prob_under_smote_tuned = grid_gb_under_smote.best_estimator_.predict_proba(X_test)[:, 1]
evaluate_model(y_test, y_pred_under_smote_tuned, y_prob_under_smote_tuned, "Gradient Boosting - Under+SMOTE (Otimizado)")

### Resultados: com ajuste de hiperparâmetros (ADASYN + Undersampling - Gradient Boosting)

In [ ]:
param_grid_under_adasyn_gb = {
    'under_sampler__sampling_strategy': [0.5, 0.7, 0.9],
    'over_sampler__n_neighbors': [3, 5, 7, 10],
    'gradientboost__n_estimators': [50, 100, 200],
    'gradientboost__learning_rate': [0.01, 0.1, 0.2],
    'gradientboost__max_depth': [3, 5, 7],
    'gradientboost__min_samples_split': [2, 5],
    'gradientboost__min_samples_leaf': [1, 3],
    'gradientboost__subsample': [0.8, 1.0],
}

In [ ]:
# --- Otimização para ADASYN + Undersampling (Gradient Boosting) ---
print("\nOtimizando Pipeline com ADASYN + Undersampling (Gradient Boosting)...")
grid_gb_under_adasyn = RandomizedSearchCV(pipeline_under_adasyn_gb, param_grid_under_adasyn_gb, n_iter=10, cv=cv_strategy,
                          scoring=scorer, n_jobs=-1, verbose=1, random_state=42)

grid_gb_under_adasyn.fit(X_train, y_train)

print(f"Melhores parâmetros para ADASYN + Undersampling (Gradient Boosting): {grid_gb_under_adasyn.best_params_}")
y_pred_under_adasyn_tuned = grid_gb_under_adasyn.best_estimator_.predict(X_test)
y_prob_under_adasyn_tuned = grid_gb_under_adasyn.best_estimator_.predict_proba(X_test)[:, 1]
evaluate_model(y_test, y_pred_under_adasyn_tuned, y_prob_under_adasyn_tuned, "Gradient Boosting - Under+ADASYN (Otimizado)")